In [4]:
import pandas as pd

In [8]:
def load_data(path="../data/csv/all.csv"):
    return pd.read_csv(path, index_col=0)

In [6]:
def vectorize_data(data_frame):
    # TODO - implement full vectorization of df
    # inputs -> number of inputs
    # outputs -> number of outputs
    # etc
    pass

In [9]:
load_data()

,jobId,read_sum,write_sum,readSyscalls_sum,writeSyscalls_sum,readReal_sum,writeReal_sum,writeCancelled_sum,rxBytes_sum,rxPackets_sum,...,mem.used,mem.active,mem.available,mem.buffers,mem.cached,mem.slab,mem.buffcache,mem.swaptotal,mem.swapused,mem.swapfree
0,-8Ip2oamm-1-10,1.640063e+07,1.865000e+03,20775.0,37.0,3.399680e+05,6.963200e+04,0.0,0.0,0.0,...,3326259200,941318144,7053393920,2768896,2302001152,590626816,2895396864,0,0,0
1,-8Ip2oamm-1-11,3.943547e+09,9.902900e+04,863138.0,1382.0,2.988442e+09,4.825088e+06,0.0,506728.0,1192.0,...,3656019968,1019281408,7034150912,2768896,2545008640,594448384,3142225920,0,0,0
2,-8Ip2oamm-1-12,2.260023e+09,6.388600e+04,561366.0,906.0,1.886495e+09,2.158592e+06,0.0,341954.0,803.0,...,3795075072,995864576,7057567744,2768896,2772025344,480731136,3255525376,0,0,0
3,-8Ip2oamm-1-13,2.284814e+09,1.477900e+05,582911.0,29318.0,4.472832e+06,3.805184e+06,0.0,354552.0,832.0,...,4242853888,1047011328,6947700736,2768896,3209531392,467460096,3679760384,0,0,0
4,-8Ip2oamm-1-14,2.754452e+09,7.098300e+04,628488.0,986.0,2.200519e+09,3.858432e+06,0.0,380100.0,894.0,...,3600007168,966131712,7087300608,2768896,2544816128,590618624,3138203648,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185815,zfuGY1yMA-1-8,4.324420e+05,9.000000e+00,332.0,9.0,0.000000e+00,3.276800e+04,0.0,0.0,0.0,...,3514421248,1118654464,32617611264,75821056,2550890496,292741120,2919452672,0,0,0
185816,zfuGY1yMA-1-80,1.420022e+07,6.100000e+01,18669.0,20.0,0.000000e+00,6.553600e+04,0.0,0.0,0.0,...,6293630976,1160232960,32576032768,85241856,5267677184,315711488,5668630528,0,0,0
185817,zfuGY1yMA-1-81,2.937060e+09,1.296316e+09,659510.0,22890.0,1.978368e+06,1.370001e+09,0.0,0.0,0.0,...,4914479104,1185255424,32551010304,109248512,3762225152,361009152,4232482816,0,0,0
185818,zfuGY1yMA-1-82,1.433393e+07,6.100000e+01,18806.0,20.0,0.000000e+00,6.553600e+04,0.0,27416.0,72.0,...,3567472640,1189007360,32547258368,81387520,2530279424,296611840,2908278784,0,0,0
